In [1]:
import numpy as np
import matplotlib.pyplot as plot

VIS_DIR = "runs/"
BENCH_DIR = "exports/"

The following two cells allow for graphing nodes by level, as well as the closest node for a single query on every level.

In [2]:
# Read graph and one query (optional) from file
GRAPH_FILE = "graph.txt"
QUERIES_FILE = "queries.txt"
PLOT_EDGES = True
PLOT_QUERY_INDEX = 3

mode = -1
level = -1
nodes_by_level = []
connections_by_level = []
with open(VIS_DIR + GRAPH_FILE, "r") as f:
    for line in f.readlines():
        line = line.strip()
        if line == "Nodes":
            mode = 0
            level = -1
            continue
        elif line == "Edges":
            mode = 1
            level = -1
            continue
        
        if mode == 0:
            if line.startswith("Level"):
                nodes_by_level.append({})
                level = int(line.split(" ")[1])
            else:
                nodes = [float(x) for x in line.split(":")[1].split(",")]
                node_id = int(line.split(":")[0])
                nodes_by_level[level][node_id] = nodes
        elif mode == 1:
            if line.startswith("Level"):
                connections_by_level.append({})
                level = int(line.split(" ")[1])
            else:
                if line[-1] == ",":
                    line = line[:-1]
                connections = [int(x) for x in line.split(":")[1].split(",")]
                node_id = int(line.split(":")[0])
                connections_by_level[level][node_id] = connections

query = None
query_neighbors = None
query_path = None
if PLOT_QUERY_INDEX >= 0:
    reading_query = -1
    with open(VIS_DIR + QUERIES_FILE, "r") as f:
        for line in f.readlines():
            line = line.strip()
            
            if line == "Query {}".format(PLOT_QUERY_INDEX):
                reading_query = 0
                continue
            if line[-1] == ",":
                line = line[:-1]

            if reading_query == 0:
                query = [float(x) for x in line.split(",")]
                reading_query = 1
            elif reading_query == 1:
                query_neighbors = [int(x) for x in line.split(",")]
                reading_query = 2
            elif reading_query == 2:
                query_path = [int(x) for x in line.split(",")]
                reading_query = -1
                break

In [ ]:
# Plot nodes as 2D points by level
EXPORT_PNG = True

for i in range(len(nodes_by_level)):
    layer_nodes = nodes_by_level[i]
    nodes = np.array(list(layer_nodes.values()))
    plot.title("Level " + str(i))
    plot.scatter(nodes[:, 0], nodes[:, 1])

    if i == 0:
        min_x, max_x = plot.gca().get_xlim()
        min_y, max_y = plot.gca().get_ylim()
    else:
        plot.xlim(min_x, max_x)
        plot.ylim(min_y, max_y)

    # Plot edges
    connections = connections_by_level[i]
    if PLOT_EDGES:
        for node_id in connections:
            for connection in connections[node_id]:
                width, height = max_x - min_x, max_y - min_y
                plot.arrow(layer_nodes[node_id][0], layer_nodes[node_id][1], layer_nodes[connection][0] - layer_nodes[node_id][0], layer_nodes[connection][1] - layer_nodes[node_id][1],
                        color='green', alpha=0.3, head_width=width / 35, head_length=height / 35, length_includes_head=True)
            
    if PLOT_QUERY_INDEX >= 0:
        plot.scatter(query[0], query[1], color='red', marker='x')
        if i > 0:
            # Plot entry point
            point = query_path[len(nodes_by_level) - 1 - i]
            plot.scatter(layer_nodes[point][0], layer_nodes[point][1], color='red', marker='o', facecolors='none')
        else:
            # Plot query neighbors
            for neighbor in query_neighbors:
                plot.scatter(layer_nodes[neighbor][0], layer_nodes[neighbor][1], color='yellow', marker='x')

    if EXPORT_PNG:
        plot.savefig(VIS_DIR + "Level " + str(i) + ".png")
    plot.show()

The following two cells allow for visualizing HNSW search on the bottommost level.

In [ ]:
# Read a specific query from file
QUERY_SEARCH_FILE = "query_search.txt"

query_search = {}
reading_query = -1
iteration = -1
with open(VIS_DIR + QUERY_SEARCH_FILE, "r") as f:
    for line in f.readlines():
        line = line.strip()
        if line[-1] == ",":
            line = line[:-1]

        if line.startswith("Iteration "):
            iteration = int(line.split(" ")[1])
            query_search[iteration] = {}
            reading_query = 0
        elif reading_query == 0:
            query_search[iteration]["visit"] = [int(x) for x in line.split(",")]
            reading_query = 1
        elif reading_query == 1:
            query_search[iteration]["cand"] = [int(x) for x in line.split(",")]
            reading_query = 2
        elif reading_query == 2:
            query_search[iteration]["found"] = [int(x) for x in line.split(",")]
            reading_query = -1

In [ ]:
# Iterate over query_search pairs
PLOT_EDGES = True
EXPORT_PNG = True
ITERATION_SKIPS = 1

for itr in query_search:
    if itr % ITERATION_SKIPS != 0 and itr != iteration:
        continue

    layer_nodes = nodes_by_level[0]
    nodes = np.array(list(layer_nodes.values()))
    plot.title("Iteration " + str(itr))
    plot.scatter(nodes[:, 0], nodes[:, 1])

    min_x, max_x = plot.gca().get_xlim()
    min_y, max_y = plot.gca().get_ylim()

    # Plot edges
    connections = connections_by_level[0]
    if PLOT_EDGES:
        for node_id in connections:
            for connection in connections[node_id]:
                width, height = max_x - min_x, max_y - min_y
                plot.arrow(layer_nodes[node_id][0], layer_nodes[node_id][1], layer_nodes[connection][0] - layer_nodes[node_id][0], layer_nodes[connection][1] - layer_nodes[node_id][1],
                        color='green', alpha=0.3, head_width=width / 35, head_length=height / 35, length_includes_head=True)
            
    # Plot query search
    for node_id in query_search[itr]["visit"]:
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='red', marker='o', facecolors='none')
    for node_id in query_search[itr]["cand"]:
        width, height = max_x - min_x, max_y - min_y
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='blue', marker='s', s=100, facecolors='none')
    for node_id in query_search[itr]["found"]:
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='yellow', marker='x')
    
    if EXPORT_PNG:
        plot.savefig(VIS_DIR + "Iteration " + str(itr) + ".png")
    plot.show()

The following cell allows for visualizating recall and distance computations for a HNSW benchmark run.

In [ ]:
QUERIES_LIST = [100]
RESULTS_LIST = [20]
GRAPH_NAME = "random_graph"
GRAPH_ID = 0

for i in range(len(QUERIES_LIST)):
    csv_name = GRAPH_NAME + "_results_{}_{}_{}.csv".format(QUERIES_LIST[i], RESULTS_LIST[i], GRAPH_ID)
    with open(BENCH_DIR + csv_name, 'r') as f:
        # Read title
        title = f.readline()

        # Read lines and convert each line to a list of floats
        graph_line = [line.split(',') for line in f.readlines()]
        graph_line = [[float(value) for value in point] for point in graph_line]

        # Graph line in plot
        graph_line = np.array(graph_line)
        plot.plot(graph_line[:,1], graph_line[:,2], 'o-')

plot.legend(["{} queries, {} results".format(QUERIES_LIST[i], RESULTS_LIST[i]) for i in range(len(QUERIES_LIST))])
plot.xlabel("Distance Computations")
plot.ylabel("Recall")
plot.title(title)
plot.show()